In [5]:
import glob
import numpy as np
import re
import os
from pathlib import Path

In [6]:

file_path = os.path.join('..','ionex_c1p','2013','c1pg1150.14i')
file = open(file_path,'r')
data = file.read()
file.close()

In [18]:
test_map = np.zeros((71,73),dtype=np.int) + np.array(list(range(1,72))).reshape(-1,1)
n=16
test_map_str = [["   "+'   '.join(map(str, arr[i:i + n])) for i in range(0, len(arr), n)] for arr in test_map]

test_map_str = ['\n'.join(map(str, l)) for l in test_map_str]

# test_map_str = ["   "+'   '.join(map(str, arr)) for arr in test_map]

# arr =test_map[0,:]
# ' '.join(map(str, arr))
# test_map_str

# map_matrix2string_ionex(test_map)

pred_maps =  np.expand_dims(np.repeat(test_map[np.newaxis,:, :], 13, axis=0), axis=0)
pred_maps = np.random.rand(1,13,71,73)
pred_maps.shape

(1, 13, 71, 73)

In [36]:
test_list = ['1','2','3','4','5','6','7','8','9','10']

n=3

output=[test_list[i:i + n] for i in range(0, len(test_list), n)]
print(output)

[['1', '2', '3'], ['4', '5', '6'], ['7', '8', '9'], ['10']]


In [37]:
data = ''
with open(file_path,'r') as f:
    line  = f.readline()
    map_count = 0
    lat_count = 0
    while line:
        data += line#+"\n"
        # if "START OF TEC MAP" in line:
        #     line  = f.readline()
        #     data += line+"\n"
        if "LAT/LON1/LON2/DLON/H" in line:
            try:
                for _ in range(5):
                    line  = f.readline()
                data += test_map_str[lat_count]+"\n"
                lat_count+=1
            except IndexError as ie:
                #print(map_count,lat_count,line)
                pass
        if "END OF TEC MAP" in line:
            # print(map_count,lat_count)
            map_count+=1
            lat_count=0

        line  = f.readline()
        if "START OF RMS MAP" in line:
            break
    while line:
        data += line#+"\n"
        line  = f.readline()
    # print(map_count)
# data



dmd_file = file_path.replace('c1p','dmd')

print(dmd_file,os.path.dirname(dmd_file))
Path(os.path.dirname(dmd_file)).mkdir(parents=True, exist_ok=True)
with open(dmd_file,'w') as f:
    f.write(data)

..\ionex_dmd\2013\dmdg1150.14i ..\ionex_dmd\2013


In [19]:

def map_matrix2string_ionex(matrix,inonex_map_columns=16):
    n=inonex_map_columns
    test_map_str = [["   "+'   '.join(map(str, arr[i:i + n])) for i in range(0, len(arr), n)] for arr in matrix]
    test_map_str = ['\n'.join(map(str, l)) for l in test_map_str]
    return test_map_str


def dmd_ionex(c1p_file_path,dmd_predicted_maps,_replace='c1p',_replace_with='dmd',debug=False):
    data = ''
    file_path = c1p_file_path
    maps = dmd_predicted_maps[0]
    with open(file_path,'r') as f:
        line  = f.readline()
        map_count = 0
        lat_count = 0
        test_map_str = map_matrix2string_ionex(maps[map_count])
        while line:
            
            data += line
            if "LAT/LON1/LON2/DLON/H" in line:
                try:
                    for _ in range(5):
                        line  = f.readline()
                    data += test_map_str[lat_count]+"\n"
                    lat_count+=1
                except IndexError as ie:
                    if debug: print('ERROR : ',map_count,lat_count,line)
                    pass
            if "END OF TEC MAP" in line:
                map_count+=1
                lat_count=0
                try:
                    test_map_str = map_matrix2string_ionex(maps[map_count])
                except:
                     if debug: print('ERROR : ',map_count,maps.shape)

            line  = f.readline()
            if "START OF RMS MAP" in line:
                break
        while line:
            data += line
            line  = f.readline()

    dmd_file = file_path.replace(_replace,_replace_with)

    if debug : print(dmd_file,os.path.dirname(dmd_file))
    Path(os.path.dirname(dmd_file)).mkdir(parents=True, exist_ok=True)
    with open(dmd_file,'w') as f:
        f.write(data)
    

dmd_ionex(file_path,pred_maps)
